In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [3]:
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

MODEL_NAME = "lightgbm"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger(f"{ CFG[MODEL_NAME]['execution']['exp_id']}.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

2024-11-16 14:55:24.779087: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 14:55:24.805894: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


exp_008


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
set seed: 46


In [4]:
import warnings

import numpy as np
import polars as pl
from IPython.display import clear_output, display
from penguinml.gbdt.lightgbm import fit_lgb, inference_lgb
from tqdm import tqdm

from const import TARGET_COLS
from feature_engineering import feature_engineer

warnings.filterwarnings("ignore")
pl.Config.set_tbl_rows(20)

train = pl.read_csv(os.path.join(CFG["dataset"]["competition_dir"], "train_features.csv"))
train = (
    train.with_columns(
        pl.col("ID").str.split_exact("_", n=1).struct.rename_fields(["sceneID", "offset"]).alias("fields")
    )
    .unnest("fields")
    .with_columns(pl.col("offset").cast(pl.Float32))
)
print(train.shape)
train.head(1)

(43371, 32)


ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,sceneID,offset
str,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f32
"""00066be8e20318869c38c66be46663…",5.701526,1.538456,-2.165777,-139.0,0.0,false,0.25,true,"""drive""",false,false,2.82959,0.032226,0.045187,6.231999,0.065895,0.107974,9.785009,0.124972,0.203649,13.485472,0.163448,0.302818,17.574227,0.174289,0.406331,21.951269,0.199503,0.485079,"""00066be8e20318869c38c66be46663…",320.0


## 特徴量生成


In [5]:
train, features = feature_engineer(train)
print(len(features))

105


## CV Split


In [6]:
train_folds = pl.read_csv(CFG["dataset"]["train_fold_path"])
train = train.join(train_folds, on="sceneID", how="left")
assert train["fold"].null_count() == 0

## Training


In [7]:
dfs = []
scores = []
for c in tqdm(sorted(TARGET_COLS, key=lambda x: x.split("_")[1], reverse=True)):
    this_df = train.filter(pl.col("target_name_original") == c)
    oof, models = fit_lgb(
        data=this_df,
        features=features,
        params=CFG[MODEL_NAME]["params"],
        target_col="target",
        fold_col="fold",
        target_type="regression",
        verbose=50000,
    )
    this_df = this_df.with_columns(pl.Series("oof", oof))
    dfs.append(this_df)

    i = int(c.split("_")[1])
    xyz = c.split("_")[0]
    mae = np.abs(this_df["oof"] - this_df["target"]).mean()
    score = {"target": c, "cv": mae} | {
        f"fold{fold}": np.abs(
            this_df.filter(pl.col("fold") == fold)["oof"] - this_df.filter(pl.col("fold") == fold)["target"]
        ).mean()
        for fold in range(5)
    }
    scores.append(score)

    clear_output()
    display(pl.DataFrame(scores).sort("target"))

train = pl.concat(dfs)
clear_output()
display(pl.DataFrame(scores).sort("target"))

target,cv,fold0,fold1,fold2,fold3,fold4
str,f64,f64,f64,f64,f64,f64
"""x_0""",0.061511,0.06091,0.060079,0.063295,0.061225,0.062046
"""x_1""",0.131055,0.130114,0.129888,0.132783,0.129897,0.132596
"""x_2""",0.223683,0.220524,0.222937,0.224705,0.222019,0.228229
"""x_3""",0.350755,0.344214,0.353736,0.354047,0.34791,0.353871
"""x_4""",0.503678,0.496198,0.511236,0.50406,0.499329,0.507566
"""x_5""",0.677177,0.665286,0.689209,0.674559,0.67423,0.682601
"""y_0""",0.031838,0.031707,0.031125,0.032146,0.031402,0.032808
"""y_1""",0.072552,0.072208,0.070447,0.073614,0.072142,0.07435
"""y_2""",0.13077,0.12929,0.128907,0.131902,0.129427,0.134322


In [8]:
mae = np.abs(train["oof"] - train["target"]).mean()
print(f"MAE: {mae}")

MAE: 0.21162359966297


In [9]:
oof_df = (
    train.select(["ID", "target_name_original", "oof"])
    .with_columns(pl.col("target_name_original"))
    .pivot(index="ID", columns="target_name_original", values="oof")
)
oof_df.head()

ID,x_5,y_5,z_5,x_4,y_4,z_4,x_3,y_3,z_3,x_2,y_2,z_2,x_1,y_1,z_1,x_0,y_0,z_0
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""00066be8e20318869c38c66be46663…",22.35674,-0.456798,0.146584,17.936957,-0.278599,0.106408,13.764888,-0.137275,0.068394,9.480743,-0.050536,0.035936,5.892587,-0.011993,0.026173,2.632299,0.004788,0.010252
"""00066be8e20318869c38c66be46663…",31.81184,-1.064219,0.125836,26.638608,-0.793587,0.105055,21.152229,-0.590548,0.130085,15.927082,-0.405887,0.068538,10.537369,-0.232908,0.0517,5.033422,-0.069232,0.016553
"""00066be8e20318869c38c66be46663…",31.707277,-0.389097,0.09333,26.699995,-0.302631,0.08712,21.053409,-0.217911,0.073546,15.576728,-0.13273,0.046212,10.079985,-0.06933,0.024713,4.741966,-0.017355,0.010333
"""000fb056f97572d384bae4f5fc1e0f…",14.126661,-0.753213,0.095061,11.369666,-0.632695,0.060883,8.930776,-0.427208,0.043425,6.061875,-0.201565,0.017495,3.630051,-0.138451,0.008878,1.57467,-0.051497,0.002661
"""000fb056f97572d384bae4f5fc1e0f…",17.446935,1.109709,0.005715,14.892217,0.757805,0.000914,11.850949,0.53762,0.00266,8.822493,0.261482,-0.001341,5.807395,0.109379,0.004992,2.76567,0.020997,0.003377


In [10]:
oof_df.write_csv(os.path.join(CFG["output_dir"], "oof.csv"))